# 중복되는 trans_id에 대한 2.거래정보, 4.분류명 정보 추가

    * 

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

import random
random.seed(1)

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.8/site-packages/tqdm/std.py:725: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


 * raw 데이터 불러오기

In [2]:
# 1. 온라인 행동 데이터
raw_data1_Online_Act = pd.read_csv('./data/data1_Online_Act.csv')

# 2. 거래 정보 데이터
raw_data2_Trade_Inform = pd.read_csv('./data/data2_Trade_Inform.csv')

# 3. 사용자 demo 정보(나이, 성별)
raw_data3_Demog = pd.read_csv('./data/data3_Demog.csv')

# 4. 상품 대중소 분류 정보
raw_data4_Item_Class = pd.read_csv('./data/data4_Item_Class.csv')

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### 2. 거래정보와 4. 분류명 정보를 결합하여 각 대분류 내에서 중분류의 거래 비율이 얼마나 되는지 확인

In [5]:
raw_data2_Trade_Inform.head(3)

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
0,21922,104999,1,A03,unknown,20190920,12:41,5990,1
1,21279,104907,4,A03,unknown,20190920,10:27,10900,1
2,39423,105124,11,A03,unknown,20190920,17:26,12900,1


In [6]:
raw_data4_Item_Class.head(3)

,pd_c,clac_nm1,clac_nm2,clac_nm3
0,1,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Automobile Oil / Additives
1,2,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Lights
2,3,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Paint


 * 결측치 확인

In [7]:
raw_data4_Item_Class.isna().sum()

pd_c        0
clac_nm1    0
clac_nm2    0
clac_nm3    2
dtype: int64

 * 2.거래정보 데이터에 4.상품분류명 데이터 병합

    * merge의 key역할을 수행하기 위해 raw_data4_Item_Class['pd_c'] data를 raw_data2와 동일한 str 형으로 변경
    
     ### 4. 데이터 병합을 위하여 상품 이름정보의 pd_c 데이터 타입 변경 (int -> str)
     * raw_data2_Trade_Inform에 있는 pd_c열의 경우, NaN과 unknown 값을 포함하기 때문에,  
       자료형을 int나 float로 변경할 수 없음(변경 시, 에러 발생)
     * 따라서 4. 상품 이름 정보의 pd_c를 str 형으로 자료형 변경
     * 그냥 object 형으로만 변경 시, 값이 같아도
         merge_raw123 에서는 pd_c = '0062'(str형)으로 인식하고,
         4. 상품이름 정보에서는 pd_c = 62(int형으로 인식해서 병합 시, 데이터 matching이 안됨

In [8]:
# 4. 상품 이름 데이터의 'pd_c' 열의 값을 4자리 숫자로 변환합니다.
# 예시) '1' => '0001', '62' => '0062', '773' => '0773' 

raw_data4_Item_Class['pd_c'] = raw_data4_Item_Class['pd_c'].astype('str')

cnt_add_zero = 0

for i in range(999):
    cnt_add_zero = 4 - len(raw_data4_Item_Class['pd_c'][i])
    add_zero = "0" * cnt_add_zero
    raw_data4_Item_Class['pd_c'][i] = add_zero + raw_data4_Item_Class['pd_c'][i]

In [9]:
merge_raw24 = pd.merge(raw_data2_Trade_Inform, raw_data4_Item_Class, on='pd_c', how='left')

In [10]:
merge_raw24

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct,clac_nm1,clac_nm2,clac_nm3
0,21922,104999,1,A03,unknown,20190920,12:41,5990,1,NaN,NaN,NaN
1,21279,104907,4,A03,unknown,20190920,10:27,10900,1,NaN,NaN,NaN
2,39423,105124,11,A03,unknown,20190920,17:26,12900,1,NaN,NaN,NaN
3,18362,104010,1,A03,unknown,20190920,09:57,9900,1,NaN,NaN,NaN
4,39423,105124,13,A03,0565,20190920,17:26,2990,1,Fruits,Imported Fruits,Bananas
...,...,...,...,...,...,...,...,...,...,...,...,...
599956,35311,2839,3,B03,0339,20190927,14:09,3500,1,Cosmetics / Beauty Care,Skin Care,Facial Masks
599957,35311,2839,4,B03,0339,20190927,14:09,3600,2,Cosmetics / Beauty Care,Skin Care,Facial Masks
599958,35311,2839,5,B03,0339,20190927,14:09,3500,1,Cosmetics / Beauty Care,Skin Care,Facial Masks
599959,35311,2839,2,B03,0339,20190927,14:09,4500,1,Cosmetics / Beauty Care,Skin Care,Facial Masks


 * Merge Data 불러오기

In [11]:
merge_table_add_nm2_all = pd.read_csv("merge_all_create_sech_clac_nm2.csv")
merge_table_add_nm2_all.drop("Unnamed: 0", axis=1, inplace=True)

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (17,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
merge_table_add_nm2_all

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,...,clnt_age,clac_nm1,clac_nm2,clac_nm3,pd_c_2,sech_kwd_over_3,sech_kwd_cat_0,sech_kwd_cat_1,sech_clac_nm1,sech_clac_nm2
0,72429,1,2,2,A03,20190919,22:09,842130,NaN,NaN,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72429,1,1,1,A03,20190919,22:09,839064,NaN,NaN,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72428,5,1,0,A01,20190918,06:37,12568,NaN,라인에디션블라우스,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72428,4,1,0,A01,20190914,23:42,243555,NaN,펜디가방,...,unknown,NaN,NaN,NaN,NaN,펜디가방,명품/쥬얼리,수입명품/해외배송,Fashion Accessories,Jewelry
4,72428,3,3,0,A01,20190914,21:50,839771,NaN,페레가모가방,...,unknown,NaN,NaN,NaN,NaN,페레가모가방,명품/쥬얼리,수입명품/해외배송,Fashion Accessories,Women's Shoes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196357,1,2,2,0,A01,20190922,14:10,56113,NaN,초등가을점퍼,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3196358,1,2,1,0,A01,20190922,14:09,41584,NaN,초등가을잠바,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3196359,1,1,3,0,A01,20190911,16:15,36140,NaN,과일바구니,...,unknown,NaN,NaN,NaN,NaN,과일바구니,여행/렌탈/금융,렌탈/배달/케어,Living / Rental Services,Living Services
3196360,1,1,2,0,A01,20190911,16:15,22432,NaN,과일선물세트 백화점,...,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 * pd_c가 nan이 아닌 갯수 : 5755개

In [13]:
len(merge_table_add_nm2_all[merge_table_add_nm2_all['pd_c'].isna() == False])

5755

 * pd_c_2가 nan이 아닌 갯수 : 57569개

In [14]:
len(merge_table_add_nm2_all[merge_table_add_nm2_all['pd_c_2'].isna() == False])

57569